**데이터셋:** AI-hub 한식이미지 데이터셋

**데이터셋 구조:** 이미지파일과 properties파일(이미지에 대한 좌표값이 있는 파일 but 전체가 있지 않음)

properties파일에 존재하는 좌표만 txt로 변화하여 사용(properties_processing.ipynb)

### 1. YOLOv8 설치

In [ ]:
!pip install ultralytics

### 2. Google Drive 마운트 및 경로 설정

In [ ]:
#Colab과 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# YOLO 폴더로 이동
%cd /content/drive/MyDrive/YOLO

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/YOLO


### 3. 데이터셋 압축 해제

In [ ]:
# YOLO 데이터셋을 저장할 폴더 생성
!mkdir -p /content/YOLO

# YOLO 폴더 안의 kfood.zip 파일을 /content/kfood로 압축 해제
!unzip /content/drive/MyDrive/YOLO/dataset.zip -d /content/YOLO/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/YOLO/dataset/output_txts/Img_137_0755.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0770.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0774.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0776.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0781.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0795.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0810.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0812.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0815.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0819.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0833.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0839.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0848.txt  
  inflating: /content/YOLO/dataset/output_txts/Img_137_0852.txt  
  inflating: /content/YOLO/dataset/outpu

In [ ]:
# 데이터셋 경로 설정
dataset_path = '/content/YOLO/dataset/kfood'
output_txts_path = '/content/YOLO/dataset/output_txts'

### 4. YOLO형식으로 데이터셋 경로 설정(image와 txt가 매칭될때만 복사)

In [ ]:
import os
import shutil

# YOLO 형식으로 변환할 폴더 경로
yolo_dataset_path = '/content/YOLO/yolo_dataset'

# 대분류 폴더 순회
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)

    if os.path.isdir(category_path):
        # 중분류 폴더 순회
        for subcategory in os.listdir(category_path):
            subcategory_path = os.path.join(category_path, subcategory)

            if os.path.isdir(subcategory_path):
                # 이미지와 properties 파일을 포함한 파일 탐색
                for img_file in os.listdir(subcategory_path):
                    if img_file.endswith('.jpg') or img_file.endswith('.png') or img_file.endswith('.jpeg'):
                        img_path = os.path.join(subcategory_path, img_file)

                        # 좌표 파일을 output_txts에서 찾기
                        base_name = os.path.splitext(img_file)[0]
                        txt_file = f'{base_name}.txt'
                        txt_file_path = os.path.join(output_txts_path, txt_file)

                        # 텍스트 파일이 있는 경우에만 복사
                        if os.path.exists(txt_file_path):
                            print(f"Copying image: {img_path}")
                            shutil.copy(img_path, yolo_dataset_path)
                            print(f"Copying label: {txt_file_path}")
                            shutil.copy(txt_file_path, yolo_dataset_path)
                        else:
                            print(f"Label not found for: {img_file}, skipping image.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Copying label: /content/YOLO/dataset/output_txts/Img_122_0777.txt
Label not found for: Img_122_0835.jpg, skipping image.
Label not found for: Img_122_0344.jpg, skipping image.
Label not found for: Img_122_0308.jpg, skipping image.
Copying image: /content/YOLO/dataset/kfood/찌개/된장찌개/Img_122_0391.jpg
Copying label: /content/YOLO/dataset/output_txts/Img_122_0391.txt
Label not found for: Img_122_0779.jpg, skipping image.
Label not found for: Img_122_0720.jpg, skipping image.
Label not found for: Img_122_0771.jpg, skipping image.
Label not found for: Img_122_0039.jpg, skipping image.
Copying image: /content/YOLO/dataset/kfood/찌개/된장찌개/Img_122_0220.jpg
Copying label: /content/YOLO/dataset/output_txts/Img_122_0220.txt
Label not found for: Img_122_0894.jpg, skipping image.
Label not found for: Img_122_0847.jpg, skipping image.
Label not found for: Img_122_0723.jpg, skipping image.
Label not found for: Img_122_0215.jpg, skipping image.
Label not found for: Img_

### 5. 데이터셋 분할
8:2 비율로 학습/검증 데이터 나눔

In [ ]:
import os
import shutil
import random

# 경로 설정
dataset_path = '/content/YOLO/yolo_dataset'
train_path = '/content/YOLO/yolo_dataset/train'
val_path = '/content/YOLO/yolo_dataset/val'

# 폴더 생성
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# 이미지 파일 리스트 가져오기
image_files = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]

# 데이터를 80:20 비율로 나누기
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# 학습 데이터와 검증 데이터로 파일 복사
for file in train_files:
    shutil.copy(os.path.join(dataset_path, file), train_path)
    # 해당 이미지에 맞는 .txt 파일도 복사
    txt_file = file.replace('.jpg', '.txt')
    if os.path.exists(os.path.join(dataset_path, txt_file)):
        shutil.copy(os.path.join(dataset_path, txt_file), train_path)

for file in val_files:
    shutil.copy(os.path.join(dataset_path, file), val_path)
    # 해당 이미지에 맞는 .txt 파일도 복사
    txt_file = file.replace('.jpg', '.txt')
    if os.path.exists(os.path.join(dataset_path, txt_file)):
        shutil.copy(os.path.join(dataset_path, txt_file), val_path)

print("데이터셋을 훈련(train)과 검증(val)으로 나누었습니다.")

데이터셋을 훈련(train)과 검증(val)으로 나누었습니다.


### 6. 데이터 구성 파일 생성
data.yaml 파일을 생성하여 YOLO 모델 학습에 필요한 데이터 정보를 설정

In [ ]:
data_yaml_content = """
train: /content/YOLO/yolo_dataset/train
val: /content/YOLO/yolo_dataset/val
nc: 150  # 총 클래스 수

names:
  - 갈비구이
  - 갈치구이
  - 고등어구이
  - 곱창구이
  - 닭갈비
  - 더덕구이
  - 떡갈비
  - 불고기
  - 삼겹살
  - 장어구이
  - 조개구이
  - 조기구이
  - 황태구이
  - 훈제오리
  - 계란국
  - 떡국_만두국
  - 무국
  - 미역국
  - 북엇국
  - 시래기국
  - 육개장
  - 콩나물국
  - 과메기
  - 양념치킨
  - 젓갈
  - 콩자반
  - 편육
  - 피자
  - 후라이드치킨
  - 갓김치
  - 깍두기
  - 나박김치
  - 무생채
  - 배추김치
  - 백김치
  - 부추김치
  - 열무김치
  - 오이소박이
  - 총각김치
  - 파김치
  - 가지볶음
  - 고사리나물
  - 미역줄기볶음
  - 숙주나물
  - 시금치나물
  - 애호박볶음
  - 경단
  - 꿀떡
  - 송편
  - 만두
  - 라면
  - 막국수
  - 물냉면
  - 비빔냉면
  - 수제비
  - 열무국수
  - 잔치국수
  - 짜장면
  - 짬뽕
  - 쫄면
  - 칼국수
  - 콩국수
  - 꽈리고추무침
  - 도라지무침
  - 도토리묵
  - 잡채
  - 콩나물무침
  - 홍어무침
  - 회무침
  - 김밥
  - 김치볶음밥
  - 누룽지
  - 비빔밥
  - 새우볶음밥
  - 알밥
  - 유부초밥
  - 잡곡밥
  - 주먹밥
  - 감자채볶음
  - 건새우볶음
  - 고추장진미채볶음
  - 두부김치
  - 떡볶이
  - 라볶이
  - 멸치볶음
  - 소세지볶음
  - 어묵볶음
  - 오징어채볶음
  - 제육볶음
  - 주꾸미볶음
  - 보쌈
  - 수정과
  - 식혜
  - 간장게장
  - 양념게장
  - 깻잎장아찌
  - 떡꼬치
  - 감자전
  - 계란말이
  - 계란후라이
  - 김치전
  - 동그랑땡
  - 생선전
  - 파전
  - 호박전
  - 곱창전골
  - 갈치조림
  - 감자조림
  - 고등어조림
  - 꽁치조림
  - 두부조림
  - 땅콩조림
  - 메추리알장조림
  - 연근조림
  - 우엉조림
  - 장조림
  - 코다리조림
  - 전복죽
  - 호박죽
  - 김치찌개
  - 닭계장
  - 동태찌개
  - 된장찌개
  - 순두부찌개
  - 갈비찜
  - 계란찜
  - 김치찜
  - 꼬막찜
  - 닭볶음탕
  - 수육
  - 순대
  - 족발
  - 찜닭
  - 해물찜
  - 갈비탕
  - 감자탕
  - 곰탕_설렁탕
  - 매운탕
  - 삼계탕
  - 추어탕
  - 고추튀김
  - 새우튀김
  - 오징어튀김
  - 약과
  - 약식
  - 한과
  - 멍게
  - 산낙지
  - 물회
  - 육회
"""

# data.yaml 파일 저장
with open('/content/YOLO/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("data.yaml 파일이 작성되었습니다.")

data.yaml 파일이 작성되었습니다.


### 7. YOLO형식에 맞게 txt파일 변환

In [ ]:
import os
from PIL import Image

# train과 val 경로 설정
train_image_path = '/content/YOLO/yolo_dataset/train/'
val_image_path = '/content/YOLO/yolo_dataset/val/'

# YOLO 형식으로 라벨 파일 변환 함수
def convert_labels(image_path):
    for label_file in os.listdir(image_path):
        if label_file.endswith('.txt'):
            label_file_path = os.path.join(image_path, label_file)

            # 이미지 파일명 추출 (label 파일과 동일한 이름을 가진 이미지 파일 찾기)
            image_file_name = label_file.replace('.txt', '.jpg')  # 이미지가 .jpg라고 가정 (.png일 경우 수정 필요)
            image_file_path = os.path.join(image_path, image_file_name)

            # 이미지 크기 가져오기
            if os.path.exists(image_file_path):
                with Image.open(image_file_path) as img:
                    image_width, image_height = img.size  # 이미지의 가로, 세로 크기 구하기

                # 파일을 읽고 라벨 형식 변환
                with open(label_file_path, 'r') as file:
                    lines = file.readlines()

                new_lines = []
                for line in lines:
                    try:
                        # 기존 라벨 형식: Img_131_0781=77,182,238,210
                        parts = line.strip().split('=')[1].split(',')
                        xmin, ymin, xmax, ymax = map(int, parts)

                        # YOLO 형식으로 변환
                        x_center = (xmin + xmax) / 2 / image_width
                        y_center = (ymin + ymax) / 2 / image_height
                        width = (xmax - xmin) / image_width
                        height = (ymax - ymin) / image_height

                        # class_id는 임시로 0으로 설정 (실제 class_id로 대체 필요)
                        new_line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                        new_lines.append(new_line)
                    except Exception as e:
                        print(f"Error processing {label_file}: {e}")

                # 수정된 라벨 파일 저장
                with open(label_file_path, 'w') as file:
                    file.writelines(new_lines)

            else:
                print(f"Image file {image_file_name} not found for label {label_file}")

# train과 val 라벨 파일 변환
convert_labels(train_image_path)
convert_labels(val_image_path)

print("train과 val 폴더의 라벨이 YOLO 형식으로 변환되었습니다.")

train과 val 폴더의 라벨이 YOLO 형식으로 변환되었습니다.


### 8. YOLOv8n 모델 학습

In [ ]:
from ultralytics import YOLO

# YOLOv8 모델 불러오기
model = YOLO('yolov8n.pt')

# 모델 학습
model.train(data='/content/YOLO/data.yaml', epochs=50, imgsz=640)

### 9. 파인튜닝 진행

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO

# 학습된 모델 불러오기 (best 모델을 사용)
model = YOLO('/content/drive/MyDrive/YOLO/runs/detect/train4/weights/best.pt')  # 경로는 학습된 모델의 저장 위치에 맞게 변경

# 검증 데이터로 성능 평가
results = model.val()  # 기본적으로 'val' 데이터를 사용하여 평가

# 결과 출력
print(results)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-080bddc2872a>", line 7, in <cell line: 7>
    results = model.val()  # 기본적으로 'val' 데이터를 사용하여 평가
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 635, in val
    validator = (validator or self._smart_load("validator"))(args=args, _callbacks=self.callbacks)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/detect/val.py", line 33, in __init__
    super().__init__(dataloader, save_dir, pbar, args, _callbacks)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/validator.py", line 96, in __init__
    self.save_dir = save_dir or get_save_dir(self.args)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 362, in get_save_dir
    save_dir = increment_path(Path(project) / name

- 라벨id값이 모두 0으로 작성되어 있어 잘못학습된 모델을 사용하니 오히려 mAP가 떨어짐 (id가 0으로 되어있는게 yolov8n_default.ipynb 진행 후 알게 됐음)
- properties파일에 존재하는 라벨만 사용한 것이 문제라 생각 → properties파일에 존재하지 않는 좌표도 포함하여 전체 이미지 사용하기(empty_labeling.ipynb의 첫번째 코드)